In [2]:
import json
import msgpack
from celery import Celery

import unibox as ub

from comfyui_router.utils import Workflow
from comfyui_router.utils import get_router_config
from comfyui_router.utils import extract_images_from_response

# Initialize Celery client to connect to the broker
broker_url = get_router_config("BROKER_URL")
app = Celery('tasks', broker=broker_url, backend='rpc://')

In [3]:
wf = Workflow("/rmt/yada/dev/training-flow/data/aestehtic_dev_cfgpp_new_workflow_v1.2.json")
wf

<Workflow with 16 nodes>
Modifiable keys: ['api_seed', 'api_positive', 'api_negative', 'api_batchsize']
Key-Value pairs: {api_seed: '1231231', api_positive: '(best quality:1.4), \n\n1girl, long hair,\n\nabsurdres, best ...}

In [4]:
overrides = {
    "api_positive": "1girl, long hair, best quality"
}
wf.update_modifiable_keys(overrides)
wf

<Workflow with 16 nodes>
Modifiable keys: ['api_seed', 'api_positive', 'api_negative', 'api_batchsize']
Key-Value pairs: {api_seed: '1231231', api_positive: '1girl, long hair, best quality', api_negative: 'lowres, (bad,...}

In [5]:
# Unique key for the request
unique_key = "debug_request_001"

# Send the request to the Celery worker
result = app.send_task('comfyui_router.tasks.process_request', args=[unique_key, dict(wf)])

# Wait for the result (blocking)
packed_response = result.get(timeout=40)  # You can adjust the timeout as needed

# Decode the MessagePack response
_, raw_response = msgpack.unpackb(packed_response, raw=False)

node_imgs = extract_images_from_response(raw_response)
node_imgs

{'9': [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=896x1152>]}

In [ ]:
node_imgs['9'][0]